# Image Classification for `Flower102` dataset using Deep Learning Architecture

Import `Flowers102` dataset. This notebook created based on PyTorch 1.11 and Torchvision 0.12.

In [ ]:
import torch
import torchvision

# add syspath
import sys
sys.path.append("../src")

print(torch.__version__)
print(torchvision.__version__)

In [ ]:
from torchvision.datasets.flowers102 import Flowers102
PATH = "../data"

Download and load the dataset based on respective partition (train, val, test)

In [ ]:
from torchvision import transforms

# imagenet mean and std
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# compose toTensor() and Normalize(mean, std)
transform_fn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((227, 227)),             # resize to 227x227 for AlexNet  
    transforms.Normalize(mean, std),
])

Define data transformation

In [ ]:
trainset = Flowers102(PATH, split="train", download=True, transform=transform_fn)
valset = Flowers102(PATH, split="val", download=True, transform=transform_fn)
testset = Flowers102(PATH, split="test", download=True, transform=transform_fn)

View sample image from dataset

In [ ]:
import random

# random number
rand_idx = random.randint(0, len(trainset))
# get random sample from trainset
sample = trainset[rand_idx]

In [ ]:
# plot sample
import matplotlib.pyplot as plt

# the image is a tensor of shape (3, 224, 224) permute to (224, 224, 3)
plt.imshow(sample[0].permute(1, 2, 0))

In [ ]:
from scipy.io import loadmat

mat = loadmat(PATH + "/flowers-102/imagelabels.mat", squeeze_me=True)
print(max(mat["labels"]), min(mat["labels"])) # label range: 1-102

In [ ]:
LABEL_NAMES = [
    "pink primrose", "hard-leaved pocket orchid", "canterbury bells",
    "sweet pea", "english marigold", "tiger lily", "moon orchid",
    "bird of paradise", "monkshood", "globe thistle", "snapdragon",
    "colt's foot", "king protea", "spear thistle", "yellow iris",
    "globe-flower", "purple coneflower", "peruvian lily", "balloon flower",
    "giant white arum lily", "fire lily", "pincushion flower", "fritillary",
    "red ginger", "grape hyacinth", "corn poppy", "prince of wales feathers",
    "stemless gentian", "artichoke", "sweet william", "carnation",
    "garden phlox", "love in the mist", "mexican aster", "alpine sea holly",
    "ruby-lipped cattleya", "cape flower", "great masterwort", "siam tulip",
    "lenten rose", "barbeton daisy", "daffodil", "sword lily", "poinsettia",
    "bolero deep blue", "wallflower", "marigold", "buttercup", "oxeye daisy",
    "common dandelion", "petunia", "wild pansy", "primula", "sunflower",
    "pelargonium", "bishop of llandaff", "gaura", "geranium", "orange dahlia",
    "pink-yellow dahlia?", "cautleya spicata", "japanese anemone",
    "black-eyed susan", "silverbush", "californian poppy", "osteospermum",
    "spring crocus", "bearded iris", "windflower", "tree poppy", "gazania",
    "azalea", "water lily", "rose", "thorn apple", "morning glory",
    "passion flower", "lotus", "toad lily", "anthurium", "frangipani",
    "clematis", "hibiscus", "columbine", "desert-rose", "tree mallow",
    "magnolia", "cyclamen", "watercress", "canna lily", "hippeastrum",
    "bee balm", "ball moss", "foxglove", "bougainvillea", "camellia", "mallow",
    "mexican petunia", "bromelia", "blanket flower", "trumpet creeper",
    "blackberry lily"
]
# show label
print(LABEL_NAMES[sample[1]-1])

Create dataloader for each dataset partition

In [ ]:
from torch.utils.data import DataLoader

batch_size = 32
num_workers = 4

train_dl = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
val_dl = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
test_dl = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

Create `AlexNet` model

In [ ]:
from models.components.alex_net import AlexNet

model = AlexNet(num_classes=102)
print(model)

Define loss function and optimizer

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

loss = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.1)

Train the `model`

In [ ]:
# train for 5 epochs
num_epochs = 5

for i in range(num_epochs):
    running_loss = 0.0
    for j, (images, labels) in enumerate(train_dl):
        # forward
        outputs = model(images)
        loss_val = loss(outputs, labels-1)  # labels-1 to convert to 0-based indexing
        # backward
        optimizer.zero_grad()   # clear gradients
        loss_val.backward()     # backpropagation
        optimizer.step()        # update weights
        # print statistics
        running_loss += loss_val.item()
        if j % 10 == 9:
            print('[%d, %5d] loss: %.3f' %
                  (i + 1, j + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')